# FAISS DB

- Facebook AI Similarity Search

In [1]:
!pip install faiss-cpu

   ---------------------------------------- 0.0/18.2 MB ? eta -:--:--
   ------------------------------------ --- 16.5/18.2 MB 94.5 MB/s eta 0:00:01
   ---------------------------------------- 18.2/18.2 MB 81.8 MB/s  0:00:00


In [ ]:
# !pip install faiss-gpu

### 영화 추천 시스템

In [3]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

In [4]:
movies = [
    {"id": 0, "title": "인셉션", "description": "잠재의식에서 다른 사람의 생각을 조작하는 스토리"},
    {"id": 1, "title": "쇼생크 탈출", "description": "죄를 지은 주인공이 쇼생크 교도소에서 희망을 잃지 않고 탈출하는 이야기"},
    {"id": 2, "title": "월-E", "description": "지구에 남은 마지막 로봇이 인간과 친구가 되는 감동적인 이야기"},
    {"id": 3, "title": "오펜하이머", "description": "핵폭탄 개발의 역사와 과학자의 내적 갈등을 다룬 영화"},
    {"id": 4, "title": "인터스텔라", "description": "우주 탐험을 통해 인류의 생존 가능성을 찾는 이야기"},
    {"id": 5, "title": "타이타닉", "description": "비극적인 배 침몰 속에서도 사랑을 그린 로맨스 드라마"},
    {"id": 6, "title": "어벤져스: 엔드게임", "description": "히어로들이 우주를 구하기 위해 마지막 전투를 벌이는 이야기"},
    {"id": 7, "title": "아이언맨", "description": "천재 공학자가 슈트를 만들어 영웅이 되는 이야기"},
    {"id": 8, "title": "스파이더맨: 노 웨이 홈", "description": "다른 차원의 스파이더맨들이 한자리에 모이는 이야기"},
    {"id": 9, "title": "닥터 스트레인지", "description": "의사가 마법사가 되어 다차원을 여행하는 이야기"},
    {"id": 10, "title": "겨울왕국", "description": "얼음을 다루는 여왕과 여동생의 사랑과 용기를 그린 애니메이션"},
    {"id": 11, "title": "라푼젤", "description": "긴 머리의 소녀가 세상 밖으로 나가는 모험 이야기"},
    {"id": 12, "title": "토이 스토리", "description": "장난감들이 주인공의 사랑을 얻기 위해 벌이는 모험"},
    {"id": 13, "title": "니모를 찾아서", "description": "아들을 잃은 아버지 물고기가 바다를 가로지르는 이야기"},
    {"id": 14, "title": "업", "description": "노인이 풍선으로 집을 띄워 여행을 떠나는 감동적인 이야기"},
    {"id": 15, "title": "주토피아", "description": "동물들이 사는 도시에서 편견을 깨고 정의를 실현하는 이야기"},
    {"id": 16, "title": "코코", "description": "죽은 자들의 세계로 간 소년이 가족의 의미를 깨닫는 이야기"},
    {"id": 17, "title": "모아나", "description": "바다를 사랑하는 소녀가 부족을 구하기 위해 항해하는 이야기"},
    {"id": 18, "title": "라라랜드", "description": "꿈과 사랑 사이에서 갈등하는 두 예술가의 이야기"},
    {"id": 19, "title": "위대한 개츠비", "description": "사랑과 욕망, 부와 허영에 대한 비극적인 이야기"},
    {"id": 20, "title": "다크 나이트", "description": "배트맨과 조커의 심리적 대결을 다룬 걸작 히어로 영화"},
    {"id": 21, "title": "조커", "description": "사회에서 소외된 한 남자가 광대로 변해가는 이야기"},
    {"id": 22, "title": "듄", "description": "사막 행성 아라키스에서 벌어지는 권력과 운명의 이야기"},
    {"id": 23, "title": "매트릭스", "description": "가상 현실과 인간의 자유의지를 탐구하는 SF 영화"},
    {"id": 24, "title": "터미네이터", "description": "인공지능이 인간을 지배하려는 미래 전쟁 이야기"},
    {"id": 25, "title": "블레이드 러너 2049", "description": "인간과 복제인간의 경계를 다룬 철학적 SF 영화"},
    {"id": 26, "title": "그랜 토리노", "description": "은퇴한 노인이 이웃 소년을 통해 변해가는 이야기"},
    {"id": 27, "title": "포레스트 검프", "description": "순수한 남자의 인생을 통해 본 미국 현대사"},
    {"id": 28, "title": "캐치 미 이프 유 캔", "description": "천재 사기꾼과 FBI 요원의 두뇌 싸움을 그린 영화"},
    {"id": 29, "title": "인사이드 아웃", "description": "감정들이 주인공의 머릿속에서 벌이는 모험"},
    {"id": 30, "title": "소울", "description": "삶의 의미를 찾아가는 재즈 피아니스트의 영혼 여행"},
    {"id": 31, "title": "가디언즈 오브 갤럭시", "description": "우주 범죄자들이 힘을 합쳐 은하를 지키는 이야기"},
    {"id": 32, "title": "앤트맨", "description": "크기를 조절할 수 있는 히어로의 활약을 다룬 영화"},
    {"id": 33, "title": "블랙 팬서", "description": "와칸다의 왕이 나라를 지키기 위해 싸우는 이야기"},
    {"id": 34, "title": "캡틴 마블", "description": "강력한 초능력을 가진 여전사의 성장과 정의의 여정"},
    {"id": 35, "title": "에이리언", "description": "우주선에서 정체불명의 생명체와 싸우는 승무원들의 이야기"},
    {"id": 36, "title": "프레데터", "description": "정글 속에서 외계 생명체와의 생존 싸움을 그린 영화"},
    {"id": 37, "title": "킹스맨", "description": "비밀 요원 조직이 세상을 구하기 위해 나서는 첩보 액션"},
    {"id": 38, "title": "미션 임파서블", "description": "불가능한 작전을 수행하는 요원들의 첩보 작전"},
    {"id": 39, "title": "존 윅", "description": "은퇴한 킬러가 복수를 위해 다시 총을 드는 액션 영화"},
    {"id": 40, "title": "테넷", "description": "시간을 역행하며 세계를 구하는 비밀 요원의 이야기"},
    {"id": 41, "title": "그린북", "description": "인종 차별 시대를 배경으로 한 감동적인 우정 이야기"},
    {"id": 42, "title": "패왕별희", "description": "중국 경극 배우들의 예술과 비극적인 운명을 그린 작품"},
    {"id": 43, "title": "살인의 추억", "description": "실화를 바탕으로 한 미제 사건을 추적하는 형사의 이야기"},
    {"id": 44, "title": "기생충", "description": "가난한 가족이 부잣집에 스며드는 사회 풍자극"},
    {"id": 45, "title": "범죄도시", "description": "형사와 조직폭력배의 거친 대결을 다룬 액션 영화"},
    {"id": 46, "title": "부산행", "description": "좀비 바이러스가 퍼진 열차 안에서 생존을 위한 싸움"},
    {"id": 47, "title": "신세계", "description": "경찰과 조직 사이의 경계를 넘나드는 언더커버의 이야기"},
    {"id": 48, "title": "올드보이", "description": "15년간 감금된 남자가 복수를 계획하는 스릴러"},
    {"id": 49, "title": "박쥐", "description": "신부가 흡혈귀로 변하면서 벌어지는 비극적인 이야기"}
]

In [ ]:
# 임베딩 모델 로드 및 임베딩 처리
descriptions = [movie['description'] for movie in movies]
desc_embed = np.array([model.encode(description) for description in descriptions], dtype=np.float32)

In [ ]:
# FAISS index 생성 및 데이터 추가
dim = desc_embed.shape[1]
idx = faiss.IndexFlatL2(dim)
idx.add(desc_embed)

In [11]:
# 질의 텍스트 임베딩
query_text = "생존"
query_embedding = np.array([model.encode(query_text)], dtype=np.float32)

In [15]:
# index 검색 : 쿼리 텍스트 임베딩 검색
top_n = 2
distances, indices = idx.search(query_embedding, k=top_n)
print(distances, indices)

[[0.5982578 0.6208725]] [[44 21]]


In [17]:
for i in range(top_n):
    movie_id = indices[0][i]
    print(f"{i+1}번째 추천: {movies[movie_id]['title']} (유사도 거리: {distances[0][i]:.2f})")

1번째 추천: 기생충 (유사도 거리: 0.60)
2번째 추천: 조커 (유사도 거리: 0.62)


In [16]:
# 검색 결과 출력
for distance, index in zip(distances[0], indices[0]):
    print(f"영화 제목: {movies[index]['title']}, 거리: {distance}")

영화 제목: 기생충, 거리: 0.5982577800750732
영화 제목: 조커, 거리: 0.6208724975585938


huggingface 에서 '한국어'를 다루는 임베딩 디비를 찾아보고 가져다가 활용할 수 있다.

### 사용자 맞춤 뉴스 추천

In [18]:
!pip install requests 
!pip install beautifulsoup4

  Using cached soupsieve-2.8-py3-none-any.whl.metadata (4.6 kB)
Using cached soupsieve-2.8-py3-none-any.whl (36 kB)

   ---------------------------------------- 2/2 [beautifulsoup4]



### 네이버 뉴스 크롤링한 뉴스데이터에서 FAISS를 활용한 추천 뉴스 만들어주기

In [29]:
import requests
from bs4 import BeautifulSoup

# 1. request -> url 요청
keyword = input("뉴스 검색 키워드 입력 : ")
url = f"https://search.naver.com/search.naver?ssc=tab.news.all&where=news&sm=tab_jum&query={keyword}"
# 입력받은 값을 검색어로 변환

response = requests.get(url)
# print(type(response))

# 2. html 응답
html = response.text

# 3. BeautifulSoup 객체 생성
bs = BeautifulSoup(html, 'html.parser')


# 4. news_contents -> 기사 내용을 받아보자.
news_contents = bs.select('.fds-news-item-list-tab > div')

news_list = [] # NewEntry 함수를 통해서 나오는 결과를 저장하기 위해 빈 리스트를 미리 만듦

for idx, news_content in enumerate(news_contents):
    title_tag = news_content.select_one('span.sds-comps-text-type-headline1')
    title = title_tag.text

    news_list.append(title)

In [30]:
news_list

['젠슨 황 앉았던 깐부치킨 테이블 \'1시간 제한\'…"당연히 이해한다"',
 "깐부치킨 '주문 폭주'...최대 피해자는 교촌? [앵커리포트]",
 "젠슨 황 덕에 주문 폭발…'깐부치킨' 결국 사상 초유의 사태",
 "[단독] 'AI 깐부' 치맥 회동, 젠슨 황 34세 딸의 작품이었다 [경주 APEC]",
 '깐부치킨 젠슨 황 효과 “배달앱 검색어 1위”… 식품업계 들썩',
 "'젠슨 황' 효과 이 정도야?…깐부치킨 1호점 결국 휴업",
 '‘깐부 사랑’ 젠슨황, 엔비디아 주식 600만주 팔았다…재산 얼마 늘었...',
 '치킨 못먹었지만 우리도 깐부…젠슨황 손잡고 황제주 등극한 이 종목',
 '이런 APEC 없었다… GD 열광, K뷰티 인증, ‘8000조’ 깐부까지',
 "'깐부 동맹' 엔비디아, 현대차에 주목한 이유는…'피지컬 AI' 존재감"]

In [31]:
# 임베딩 모델 로드 및 임베딩 처리
import faiss
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer('all-MiniLM-L6-v2')
news_embed = np.array([model.encode(news) for news in news_list], dtype=np.float32)

# FAISS 인덱스 생성 및 데이터 추가
dim = news_embed.shape[1]
idx = faiss.IndexFlatL2(dim)
idx.add(news_embed)

# 질의 텍스트 임베딩
query_text = input("뉴스 검색 키워드 입력 :")
top_n = input("추천 뉴스 개수 입력 :")
query_embedding = np.array([model.encode(query_text)], dtype=np.float32)

# index 검색
top_n = int(top_n)
distances, indices = idx.search(query_embedding, k=top_n)

# 검색 결과 출력
for i in range(top_n):
    print(f"{i+1}번째 추천 뉴스: {news_list[indices[0][i]]} (유사도 거리: {distances[0][i]:.2f})")

1번째 추천 뉴스: '젠슨 황' 효과 이 정도야?…깐부치킨 1호점 결국 휴업 (유사도 거리: 0.19)
2번째 추천 뉴스: 젠슨 황 앉았던 깐부치킨 테이블 '1시간 제한'…"당연히 이해한다" (유사도 거리: 0.60)
3번째 추천 뉴스: 깐부치킨 젠슨 황 효과 “배달앱 검색어 1위”… 식품업계 들썩 (유사도 거리: 0.60)
4번째 추천 뉴스: 젠슨 황 덕에 주문 폭발…'깐부치킨' 결국 사상 초유의 사태 (유사도 거리: 0.70)
